In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
    [torchvision.transforms.Resize((128,960))])
#     [torchvision.transforms.CenterCrop((160,1072))])


In [ ]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/512_128/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/512_128/test/{filename}')
            
        return spec


    def inv_stem(self,x):
        x1 = x.view(24, 20, 16, 16)
        y = torch.zeros(384, 384, dtype=x.dtype)
        for i in range(24):
            for j in range(20):
                y[i*16:(i+1)*16, j*16:(j+1)*16] = x1[i, j]
        return y

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
        
#         samples = self.inv_stem(samples).unsqueeze(0)
        
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [ ]:
a,b = next(iter(POGFiles(df_train,transform=None)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

In [ ]:
# filename = '000000.npy'
# a = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_448_160/train/{filename}')
# b = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/train/{filename}')

#### AST: https://github.com/YuanGongND/ast/blob/master/src/models/ast_models.py

In [ ]:
NCLASS = 19

In [ ]:
import timm
timm.list_models()
# timm.create_model('deit_base_patch16_384',in_chans=1,pretrained=False)

In [ ]:
timm.create_model('resnext50d_32x4d',in_chans=1,pretrained=False)

In [ ]:
NCLASS = df_train['genre_id'].nunique()
NCLASS

In [ ]:
import time

In [ ]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnext50d_32x4d_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #BiTemperedLogisticLoss()#CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 32
        n_epochs = 20

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnext50d_32x4d',in_chans=1,pretrained=False)
        #get_model(resnet34)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)
    
        f1_score = F1Score(average="micro")
        
        
        
        ### MixUp() in callbacks
        opt = ranger
        learn = Learner(dls, model,opt_func=opt,loss_func=crit,metrics=[f1_score],cbs=[MixUp(0.1),SaveModelCallback('f1_score', every_epoch=True),
                       CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
        
        learn.fit_flat_cos(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
if __name__=='__main__':
    run()

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
del _

In [ ]:
gc.collect()

#### Prog resize

#### Dataset

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((160,1072))])
    [torchvision.transforms.CenterCrop((160,1072))])


In [ ]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [ ]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/test/{filename}')
            
        return spec


    def inv_stem(self,x):
        x1 = x.view(24, 20, 16, 16)
        y = torch.zeros(384, 384, dtype=x.dtype)
        for i in range(24):
            for j in range(20):
                y[i*16:(i+1)*16, j*16:(j+1)*16] = x1[i, j]
        return y

    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
        
#         samples = self.inv_stem(samples).unsqueeze(0)
        
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
a,b = next(iter(POGFiles(df_train,transform=tfms)))
a.shape

In [ ]:
def runProgResize():
    oof = df_train.copy()
    oversampleTrain = False
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnext50d_32x4d_MonoSpec_Hop448_Mels160'
        kernel_type_prev = 'resnext50d_32x4d_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #BiTemperedLogisticLoss()#CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 24
        n_epochs = 15

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnext50d_32x4d',in_chans=1,pretrained=False)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)
    
    
        f1_score = F1Score(average="micro")
        
        opt = ranger
        learn = Learner(dls, model,opt_func=opt,loss_func=crit,metrics=[f1_score],cbs=[MixUp(0.2),SaveModelCallback('f1_score', every_epoch=True),
                       CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
                
        weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
        learn.model.load_state_dict(weights,strict=False)
        
        learn.fit_flat_cos(n_epochs, 1e-2,wd=1e-3)
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [ ]:
if __name__=='__main__':
    runProgResize()

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
del _

In [ ]:
gc.collect()

### OOF

In [ ]:
kernel_type = 'resnext50d_32x4d_MonoSpec_Hop448_Mels160'

In [ ]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'resnext50d_32x4d_MonoSpec_Hop448_Mels160'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [ ]:
OOFOut['ID']

In [ ]:
OOFOut.tail()

In [ ]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [ ]:
OOFOut.head()

In [ ]:
OOFOut['genre_id'].max()

In [ ]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [ ]:
OOF_preds

In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

### Mode based sub

In [ ]:
kernel_type

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnest50d_MonoSpec_Hop448_Mels160_FTune',
                    'resnest50d_MonoSpec_Hop448_Mels160',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels.csv',index=False)

### Mode based sub: check OOF Score

In [44]:
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')

In [45]:
import itertools

In [61]:
iters = ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160']


combs = list(itertools.combinations(iters, 3))
# combs = [i for i in combs if 'b1' in j for j in i]
len(combs)

286

In [62]:
from tqdm import tqdm
best = 0
for comb in tqdm(combs):
    out = pd.DataFrame()
    for kernels in comb:
#         print('Kernel:', kernels)
        OOFOut = pd.DataFrame()
        for fld in range(5):
            fold_num = fld
            kernel_type= kernels
            OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
            ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
            OOF = ([item for sublist in OOF for item in sublist])
            OOF = pd.DataFrame(OOF)
            OOF['ID'] = ID
            OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

        OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
        OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
        out = pd.concat([out,OOF_preds],1)
        out['genre_id'] = OOFOut['genre_id']

    out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
    f1_score(out['genre_id'],out['preds'],average="micro")
    scr = f1_score(out['genre_id'],out['preds'],average="micro")
    if scr > best:
        print(f1_score(out['genre_id'],out['preds'],average="micro"))
        best = scr
        print(comb)
    print('-'*75)

  0%|          | 1/286 [00:05<26:24,  5.56s/it]

0.5757484428370504
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160')
---------------------------------------------------------------------------


  1%|          | 2/286 [00:11<26:06,  5.52s/it]

0.5785111512959614
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128')
---------------------------------------------------------------------------


  1%|          | 3/286 [00:16<25:38,  5.43s/it]

...

---------------------------------------------------------------------------


100%|█████████▉| 285/286 [25:56<00:05,  5.42s/it]

---------------------------------------------------------------------------


100%|██████████| 286/286 [26:02<00:00,  5.46s/it]

---------------------------------------------------------------------------


In [63]:
best

0.6007132810930279

In [ ]:
# best: ('resnest50d_MonoSpec_Hop448_Mels160_FTune', 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 'resnext50d_32x4d_MonoSpec_Hop448_Mels160')

In [64]:
iters = ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160']


combs = list(itertools.combinations(iters, 4))
combs

[('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'tf_efficientnet_b1_MonoSpec_Hop512_Mels128'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'densenet121d_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnest26d_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'seresnet50t_MonoSpec_Hop448_Mels16

In [65]:
len(combs)

715

In [66]:
from tqdm import tqdm
best = 0
for comb in tqdm(combs):
    out = pd.DataFrame()
    for kernels in comb:
        OOFOut = pd.DataFrame()
        for fld in range(5):
            fold_num = fld
            kernel_type= kernels
            OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
            ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
            OOF = ([item for sublist in OOF for item in sublist])
            OOF = pd.DataFrame(OOF)
            OOF['ID'] = ID
            OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

        OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
        OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
        out = pd.concat([out,OOF_preds],1)
        out['genre_id'] = OOFOut['genre_id']

    out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
    f1_score(out['genre_id'],out['preds'],average="micro")
    scr = f1_score(out['genre_id'],out['preds'],average="micro")
    if scr > best:
        print(f1_score(out['genre_id'],out['preds'],average="micro"))
        best = scr
        print(comb)
    print('-'*75)

  0%|          | 1/715 [00:06<1:12:47,  6.12s/it]

0.5794153104279687
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160', 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128')
---------------------------------------------------------------------------


  0%|          | 2/715 [00:11<1:10:47,  5.96s/it]

---------------------------------------------------------------------------


  0%|          | 3/715 [00:17<1:09:35,  5.86s/it]

...

---------------------------------------------------------------------------


100%|█████████▉| 714/715 [1:07:06<00:05,  5.57s/it]

---------------------------------------------------------------------------


100%|██████████| 715/715 [1:07:12<00:00,  5.64s/it]

---------------------------------------------------------------------------


In [68]:
# best 60.35 : ('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 'resnext50d_32x4d_MonoSpec_Hop448_Mels160')

In [70]:
out = pd.DataFrame()
for kernels in ['eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 'resnext50d_32x4d_MonoSpec_Hop448_Mels160'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

0.6035262206148282

In [71]:
iters = ['eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160',
                'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                'tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
                'densenet121d_MonoSpec_Hop448_Mels160_FTUNE',
                'resnest26d_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE',
                'ecaresnet50t_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160']


combs = list(itertools.combinations(iters, 5))
combs

[('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
  'tf_efficientnet_b1_MonoSpec_Hop512_Mels128'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
  'densenet121d_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
  'resnest26d_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels160',
  'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
  'ecaresnet50t_MonoSpec_Hop448_Mels160_FTUNE'),
 ('eb1_MonoSpec_Hop448_Mels160',
  'resnest50d_MonoSpec_Hop448_Mels160_FTune',
  'resnest50d_MonoSpec_Hop448_Mels1

In [72]:
len(combs)

1287

In [73]:
from tqdm import tqdm
best = 0
for comb in tqdm(combs):
    out = pd.DataFrame()
    for kernels in comb:
        OOFOut = pd.DataFrame()
        for fld in range(5):
            fold_num = fld
            kernel_type= kernels
            OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
            ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
            OOF = ([item for sublist in OOF for item in sublist])
            OOF = pd.DataFrame(OOF)
            OOF['ID'] = ID
            OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

        OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
        OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
        out = pd.concat([out,OOF_preds],1)
        out['genre_id'] = OOFOut['genre_id']

    out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
    f1_score(out['genre_id'],out['preds'],average="micro")
    scr = f1_score(out['genre_id'],out['preds'],average="micro")
    if scr > best:
        print(f1_score(out['genre_id'],out['preds'],average="micro"))
        best = scr
        print(comb)
    print('-'*75)

  0%|          | 1/1287 [00:05<2:08:24,  5.99s/it]

0.580168776371308
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160', 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128', 'tf_efficientnet_b1_MonoSpec_Hop512_Mels128')
---------------------------------------------------------------------------


  0%|          | 2/1287 [00:11<2:06:46,  5.92s/it]

0.5810227044404259
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160', 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128', 'densenet121d_MonoSpec_Hop448_Mels160_FTUNE')
---------------------------------------------------------------------------


  0%|          | 3/1287 [00:17<2:05:30,  5.86s/it]

...

---------------------------------------------------------------------------


100%|█████████▉| 1286/1287 [2:06:22<00:05,  5.89s/it]

---------------------------------------------------------------------------


100%|██████████| 1287/1287 [2:06:28<00:00,  5.90s/it]

---------------------------------------------------------------------------


###### best: 60.49; ('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain', 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 'resnext50d_32x4d_MonoSpec_Hop448_Mels160')

In [86]:
out = pd.DataFrame()
for kernels in [
                'eb1_MonoSpec_Hop448_Mels160', 
                'resnest50d_MonoSpec_Hop448_Mels160_FTune', 
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain', 
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160',
               'seresnet50t_MonoSpec_Hop448_Mels160_FTUNE'
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

0.601918826602371

In [83]:
out

,0,genre_id,0,0,0,0,0,preds
0,3,7.0,3,1,3,1,3.0,3
1,9,9.0,9,9,9,9,9.0,9
2,0,0.0,0,0,0,0,0.0,0
3,4,4.0,4,4,4,4,4.0,4
4,3,3.0,3,3,3,3,3.0,3
...,...,...,...,...,...,...,...,...
19903,4,NaN,9,9,15,1,NaN,9
19904,3,NaN,9,3,9,9,NaN,9
19905,1,NaN,1,1,1,1,NaN,1
19906,0,NaN,0,0,0,0,NaN,0


In [43]:
from tqdm import tqdm
best = 0
for comb in tqdm(combs):
    out = pd.DataFrame()
    for kernels in comb:
        OOFOut = pd.DataFrame()
        for fld in range(5):
            fold_num = fld
            kernel_type= kernels
            OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
            ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
            OOF = ([item for sublist in OOF for item in sublist])
            OOF = pd.DataFrame(OOF)
            OOF['ID'] = ID
            OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
    out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
    scr = f1_score(out['genre_id'],out['preds'],average="micro")
    if scr > best:
        print(f1_score(out['genre_id'],out['preds'],average="micro"))
        best = scr
        print(comb)
    print('-'*75)

100%|██████████| 1/1 [00:03<00:00,  3.88s/it]

0.5651999196302994
('eb1_MonoSpec_Hop448_Mels160', 'resnest50d_MonoSpec_Hop448_Mels160_FTune', 'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain', 'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160', 'resnext50d_32x4d_MonoSpec_Hop448_Mels160')
---------------------------------------------------------------------------


### Mode based sub

In [ ]:
kernel_type

In [ ]:
kernel_type ='resnext50d_32x4d_MonoSpec_Hop448_Mels160'

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in [
                'eb1_MonoSpec_Hop448_Mels160',
                'resnest50d_MonoSpec_Hop448_Mels160_FTune',
                'resnest50d_MonoSpec_Hop448_Mels160_NoisyTrain',
                'ecaresnext50t_32x4d_MonoSpec_Hop448_Mels160',
                'resnext50d_32x4d_MonoSpec_Hop448_Mels160'
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_5bestmodels.csv',index=False)

In [ ]:
kernel_type

### PL

In [ ]:
kernel_type_upd = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128' 

In [ ]:
fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/fold_{fld}.csv')
fname

In [ ]:
preds_out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    probs = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/test_fold_{fld}_probs.npy',allow_pickle=True)
    p = pd.DataFrame([item.numpy().ravel() for sublist in probs for item in sublist])
    preds_out += p.values/5

In [ ]:
preds_out.shape

In [ ]:
preds_out['song_id'] = fname['song_id']

In [ ]:
preds_out.head()

In [ ]:
preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0]

In [ ]:
PL = preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0].reset_index(drop=True)

In [ ]:
PL.to_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv')

In [ ]:
PL.head()

In [ ]:
np.argmax(PL.iloc[:,:-1].values,1)

### End ###